In [16]:
import sys

import cplex
from cplex.callbacks import UserCutCallback, LazyConstraintCallback

In [17]:
# ----------------------------------------------------------------------------
# IBM CPLEX の MIPLIB 3.0 モデル `noswot.mps` を解くスクリプト
# - ユーザーカット（User Cut）を `UserCutCallback` を使用して追加
# - 遅延制約（Lazy Constraint）を `LazyConstraintCallback` を使用して追加
# ----------------------------------------------------------------------------

In [18]:
# ----------------------------------------------------------------------------
# ユーザーカット (User Cut Callback) の定義
# - 分枝限定（Branch-and-Bound）中に、ユーザーカットを追加する
# - 目的: モデルを強化し、探索の効率を向上させる
# ----------------------------------------------------------------------------


class MyCut(UserCutCallback):
    def __init__(self, env):
        UserCutCallback.__init__(self, env)
        self.initcuts()  # カットの初期化

    def initcuts(self):
        """カットの定義（noswot 問題用の有効なカットを設定）"""
        lhs = []  # 左辺（制約の係数）
        rhs = []  # 右辺（制約の上限値）

        # カット1: X21 - X22 <= 0
        lhs.append(cplex.SparsePair(ind=["X21", "X22"], val=[1.0, -1.0]))
        rhs.append(0.0)

        # カット2: X22 - X23 <= 0
        lhs.append(cplex.SparsePair(ind=["X22", "X23"], val=[1.0, -1.0]))
        rhs.append(0.0)

        # カット3: X23 - X24 <= 0
        lhs.append(cplex.SparsePair(ind=["X23", "X24"], val=[1.0, -1.0]))
        rhs.append(0.0)

        # その他のカット（リソース制約）
        lhs.append(
            cplex.SparsePair(
                ind=[
                    "X11",
                    "X21",
                    "X31",
                    "X41",
                    "X51",
                    "W11",
                    "W21",
                    "W31",
                    "W41",
                    "W51",
                ],
                val=[2.08, 2.98, 3.47, 2.24, 2.08, 0.25, 0.25, 0.25, 0.25, 0.25],
            )
        )
        rhs.append(20.25)

        self.lhs = lhs  # 左辺のリストを保存
        self.rhs = rhs  # 右辺のリストを保存

    def __call__(self):
        """カットを適用する処理"""
        lhs = self.lhs
        rhs = self.rhs
        nCuts = len(rhs)

        for i in range(nCuts):
            # 現在のカットの評価値を計算
            act = sum(
                lhs[i].val[k] * self.get_values(lhs[i].ind[k])
                for k in range(len(lhs[i].ind))
            )

            # 制約を満たしていない場合、新しいカットを追加
            if act > rhs[i] + 1e-6:
                self.add(cut=lhs[i], sense="L", rhs=rhs[i])  # "L" は <= の意味

In [19]:
# ----------------------------------------------------------------------------
# 遅延制約 (Lazy Constraint Callback) の定義
# - CPLEX の探索中に、整数解が見つかった場合に追加
# ----------------------------------------------------------------------------


class MyLazy(LazyConstraintCallback):
    def __call__(self):
        """遅延制約を適用する処理"""
        indices = ["W11", "W12", "W13", "W14", "W15"]  # 制約を適用する変数
        act = sum(self.get_values(i) for i in indices)  # 変数の現在の値を取得

        if act > 3.01:  # 制約違反をチェック
            self.add(
                constraint=cplex.SparsePair(ind=indices, val=[1.0] * 5),  # 制約の左辺
                sense="L",  # "L" は <= の意味
                rhs=3.0,  # 制約の右辺
            )


# ----------------------------------------------------------------------------
# CPLEX ソルバーの結果を表示する関数
# ----------------------------------------------------------------------------


def solve_and_report(c):
    """MIP を解き、結果を表示する"""
    c.solve()  # CPLEX で最適化を実行

    # 解の状態と最適値を表示
    print(
        "Solution status =",
        c.solution.get_status(),
        ":",
        c.solution.status[c.solution.get_status()],
    )
    print("Objective value =", c.solution.get_objective_value())

    # 非ゼロの変数値を表示
    values = c.solution.get_values()
    for i, x in enumerate(values):
        if abs(x) > 1.0e-10:
            print(
                "Column %3d (%5s):  Value = %17.10g" % (i, c.variables.get_names(i), x)
            )


# ----------------------------------------------------------------------------
# メイン関数（CPLEX を使って `noswot.mps` を解く）
# ----------------------------------------------------------------------------


def admipex5():
    """MPS ファイル `noswot.mps` を CPLEX で解く"""
    c = cplex.Cplex("./data/noswot.mps")  # モデルをロード

    # ログの出力設定（Jupyter Notebook では省略可能）
    c.set_log_stream(sys.stdout)
    c.set_results_stream(sys.stdout)

    # 分枝限定法の探索方法を "traditional" に設定（カットの適用を有効化）
    c.parameters.mip.strategy.search.set(
        c.parameters.mip.strategy.search.values.traditional
    )

    # ログの出力間隔を 1000 ノードごとに設定
    c.parameters.mip.interval.set(1000)

    # 既存の情報を使用しない（最適化のスタートをリセット）
    c.parameters.advance.set(0)

    # ユーザーカットの適用（モデルを強化）
    c.register_callback(MyCut)
    solve_and_report(c)

    # 遅延制約の適用（最適解の制約をチェック）
    c.register_callback(MyLazy)
    solve_and_report(c)

    # ユーザーカットを解除
    c.unregister_callback(MyCut)
    solve_and_report(c)

In [20]:
admipex5()


Selected objective sense:  MINIMIZE
Selected objective  name:  1
Selected RHS        name:  RHS
Selected bound      name:  LINDOBND
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Advance                                 0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Strategy_Search                     1
CPXPARAM_MIP_Interval                            1000
Legacy callback                                  UD
Tried aggregator 2 times.
MIP Presolve eliminated 7 rows and 5 columns.
MIP Presolve modified 57 coefficients.
Aggregator did 3 substitutions.
Reduced MIP has 172 rows, 120 columns, and 685 nonzeros.
Reduced MIP has 75 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.69 ticks)
Found incumbent of value -6.000000 after 0.00 sec. (1.46 ticks)
Probing fixed 0 vars, tightened 20 bounds.
Probing time = 0.01 sec. (0.08 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve modified 10 coefficients.
Reduced MIP ha